In [5]:
import numpy as np
np.random.seed(11)
import tensorflow as tf
tf.random.set_seed(11)

In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.utils import np_utils, load_img, img_to_array

In [7]:
from sklearn.datasets import load_files
from tqdm import tqdm

## Load images

In [8]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 102)
    return files, targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('./Data/Datasplit/train')
valid_files, valid_targets = load_dataset('./Data/Datasplit/valid')
test_files, test_targets = load_dataset('./Data/Datasplit/test')

## Convert images to float

In [9]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

## Build a CNN model from scratch

In [21]:
train_tensors = paths_to_tensor(train_files)
valid_tensors = paths_to_tensor(valid_files)
test_tensors = paths_to_tensor(test_files)

100%|███████████████████████████████████████| 6149/6149 [01:09<00:00, 88.44it/s]


In [22]:
cnn = Sequential()
cnn.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(224, 224, 3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Dropout(0.3))
cnn.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(102, activation='softmax'))
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 112, 112, 64)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 32)      18464     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 56, 56, 16)       

In [23]:
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets), 
        epochs=10, batch_size=20, verbose=0)

In [24]:
cnn.evaluate(train_tensors, train_targets)
cnn.evaluate(valid_tensors, valid_targets)
cnn.evaluate(test_tensors, test_targets)

193/193 [==============================] - 121s 624ms/step - loss: 4.5455 - accuracy: 0.0259


[4.545535087585449, 0.025857863947749138]

In [25]:
# get prediction
flower_preds = [np.argmax(cnn.predict(np.expand_dims(tensor, axis=0), verbose=0)) for tensor in test_tensors]

In [26]:
# another way to get accuracy
test_accuracy = 100*np.sum(np.array(flower_preds)==np.argmax(test_targets, axis=1))/len(flower_preds)
print(f'Test accuracy: {test_accuracy:.2f}%')

Test accuracy: 2.59%
